In [3]:
import json

import asyncio
import aioredis

import nest_asyncio
nest_asyncio.apply()

import rx
from rx.subject import Subject
from rx.core import Observer, Observable, ConnectableObservable, typing
from rx.disposable import Disposable, CompositeDisposable
from rx import operators as ops

from dataclasses import dataclass, field
from typing import Type, Any, List, Tuple, Callable, Optional, Union, Awaitable, Coroutine, NamedTuple

import mape

In [4]:
from mape.redis import PubObserver, SubObservable
from mape.utils import LogObserver
from mape.typing import Message

from dataclasses import dataclass, field

@dataclass
class PayloadTest:
    name: str = 'ciao'
    number: int = 6
    numbers: List = field(default_factory=lambda: [1, 2, 3])

ciao = SubObservable(["A", "B"])
a = ciao.subscribe(LogObserver("A, B"))

ciao = SubObservable("B")
b = ciao.subscribe(LogObserver("B"))

pub_observer_a = PubObserver("A")
pub_observer_a.on_next(Message(value='a', src='src.A'))
pub_observer_a.on_next('a')
# pub_observer.on_error(Message(value="error", src='src2'))
# pub_observer.on_completed()

await asyncio.sleep(4)
# pub_observer.dispose()

print("B")

pub_observer_b = PubObserver("B")
pub_observer_b.on_next(Message(value=PayloadTest(), src='src.B'))
pub_observer_b.on_next(20)
# pub_observer.on_error(Message(value="error", src='src20'))
# pub_observer.on_completed()

await asyncio.sleep(4)
pub_observer_b.on_next('b1')
await asyncio.sleep(0.1)
pub_observer_b.on_next('b2')
await asyncio.sleep(0)
pub_observer_b.on_next('b3')
pub_observer_a.on_next('a1')
await asyncio.sleep(10)
pub_observer_a.on_next('a2')
await asyncio.sleep(1)

a.dispose()
b.dispose()

(9672) on next: Message(a, src.A, None, 0, 16:45:17.929) | A, B
(9672) on next: a | A, B
B
(9672) on next: Message(PayloadTest(name='ciao', number=6, numbers=[1, 2, 3]), src.B, None, 0, 16:45:21.931) | A, B
(8537) on next: Message(PayloadTest(name='ciao', number=6, numbers=[1, 2, 3]), src.B, None, 0, 16:45:21.931) | B
(9672) on next: 20 | A, B
(8537) on next: 20 | B
(8537) on next: b1 | B
(9672) on next: b1 | A, B
(9672) on next: b2 | A, B
(8537) on next: b2 | B
(9672) on next: b3 | A, B
(8537) on next: b3 | B
(9672) on next: a1 | A, B
(9672) on next: a2 | A, B


In [4]:
""" MANAGE TASK EXCEPTION """

import asyncio
import aioredis
import mape

import nest_asyncio
nest_asyncio.apply()

async def test_raise_unmanaged(param=None):
    while True:
        print(param)
        raise Exception('unmanaged')
        await asyncio.sleep(2)

@mape.utils.log_task_exception
async def test_raise_unmanaged_wrapped(param=None):
    while True:
        print(param)
        raise Exception('wrapped')
        await asyncio.sleep(2)

task = asyncio.create_task(test_raise_unmanaged('silent exception'))
await asyncio.sleep(4)
print("cancel")
task.cancel()

task = asyncio.create_task(mape.utils.task_exception(test_raise_unmanaged_wrapped('wrapped')))
await asyncio.sleep(4)
print("cancel")
task.cancel()

task = asyncio.create_task(test_raise_unmanaged_wrapped('decorated'))
await asyncio.sleep(4)
print("cancel")
task.cancel()

await asyncio.sleep(4)

silent exception


wrapped
Traceback (most recent call last):
  File "/home/muttley/Projects/SecondLevelDegrees/courses/Thesis/source/playground/mape/utils.py", line 129, in task_exception
    return await awaitable
  File "/tmp/ipykernel_176147/836816186.py", line 18, in test_raise_unmanaged_wrapped
    raise Exception('wrapped')
Exception: wrapped


cancel
wrapped


wrapped
Traceback (most recent call last):
  File "/home/muttley/Projects/SecondLevelDegrees/courses/Thesis/source/playground/mape/utils.py", line 129, in task_exception
    return await awaitable
  File "/tmp/ipykernel_176147/836816186.py", line 18, in test_raise_unmanaged_wrapped
    raise Exception('wrapped')
Exception: wrapped


cancel
decorated
cancel
